In [41]:
import pandas as pd
import cv2
import sys
import os
from collections import defaultdict
import time
import torch
from torch.optim import Adam
from ultralytics import YOLO, RTDETR
import torchvision.models as models
import torch.nn as nn
# Agregar el root del proyecto al path
sys.path.append(os.path.abspath("../.."))

from src.main import load_config
from scripts.get_stats import *


--- GET STATS: Analizando Dataset Global (data\processed\all_labels) ---
 No existe data\processed\all_labels. Ejecuta data_processing primero.


ValueError: not enough values to unpack (expected 3, got 0)

# YOLO

In [ ]:
# ===== LÓGICA PRINCIPAL DE ENTRENAMIENTO =====
def train():
    config = load_config()
    root_path = config['root_path']
    # -- Rutas --
    # splits_path = Path(config['paths']['splits_path'])
    # dataset_yaml = splits_path / "dataset.yaml"
    dataset_yaml = "dataset.yaml"
    #Output
    # output_dir = root_path / "models" / "artifacts"
    output_dir = "data/models/artifacts/yolo_v8"  # Concatenación simple para evitar problemas de Path en Windows
    #Crear output_dir si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"root_path: {root_path}")
    print(f"dataset_yaml: {dataset_yaml}")
    print(f"output_dir: {output_dir}")

    # Inicializar modelo YOLOv8s preentrenado
    model = YOLO('yolov8s.pt')

    # Configurar PARAMETROS DE ENTRENAMIENTO
    epochs = config['training']['epochs']
    batch_size = config['training']['batch_size']
    device = 0 if torch.cuda.is_available() else "cpu"
    # Iniciar entrenamiento
    
    results = model.train(
        data=dataset_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=640,
        project=str(output_dir),
        name="yolo_run",
        exist_ok=True,
        pretrained=True,
        plots=True,
        device=device,
        workers=4
    )

    

In [34]:
train()

root_path: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11
dataset_yaml: dataset.yaml
output_dir: data/models/artifacts/
New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.11  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, m

# RTDETR

In [36]:
def get_simple_cnn(num_classes=3):
    # Usamos ResNet18 como backbone de una CNN simple para clasificación
    model = models.resnet18(pretrained=True)
    # Ajustamos la última capa para nuestras clases (Standing, Eating, Lying) [cite: 100, 105]
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

In [37]:
def train_rtdetr():
    config = load_config()
    dataset_yaml = "dataset.yaml"
    output_dir = "models/artifacts/rtdetr"
    os.makedirs(output_dir, exist_ok=True)

    # Cargamos la variante preentrenada de RT-DETR-L
    model = RTDETR('rtdetr-l.pt')

    results = model.train(
        data=dataset_yaml,
        epochs=config['training']['epochs'],
        batch=config['training']['batch_size'],
        imgsz=640,
        project=output_dir,
        name="experiment",
        device=0 if torch.cuda.is_available() else "cpu",
        pretrained=True
    )

In [38]:
train_rtdetr()

New https://pypi.org/project/ultralytics/8.4.14 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.11  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=experiment, nbs=64, n

c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       1/50      24.2G      1.053      1.215     0.6102         56        640: 100% ━━━━━━━━━━━━ 74/74 7.1s/it 8:447.5s5s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.0s/it 7.3s0.5ss
                   all        438       5399      0.658      0.497      0.467      0.221

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


       2/50      23.6G     0.5899     0.5535     0.2094         36        640: 100% ━━━━━━━━━━━━ 74/74 3.6s/it 4:290.6ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 2.5it/s 2.9s0.5s
                   all        438       5399      0.772      0.604      0.611      0.304

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

      48/50      23.8G     0.2804     0.3379     0.1042         22        640: 100% ━━━━━━━━━━━━ 74/74 21.5s/it 26:28.3s0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.1s/it 7.5s0.7ss
                   all        438       5399      0.787      0.782      0.782      0.456

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      49/50        24G     0.2764      0.333     0.1038         21        640: 100% ━━━━━━━━━━━━ 74/74 4.0s/it 4:551.1ss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 2.0it/s 3.5s0.6ss
                   all        438       5399      0.782      0.788      0.785      0.453

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      50/50      23.9G     0.2502     0.3332     0.0924        328        640: 0% ──────────── 0/74  3.0s

c:\Bureau\Proyectos\DATAGIA\Modelo-CU11\env\Lib\site-packages\torch\autograd\graph.py:825: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\Context.cpp:95.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      50/50      23.9G     0.2724     0.3301     0.1014         25        640: 100% ━━━━━━━━━━━━ 74/74 10.9s/it 13:24.3s4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.1s/it 7.5s0.7ss
                   all        438       5399      0.785      0.776      0.785      0.455

50 epochs completed in 5.872 hours.
Optimizer stripped from C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\runs\detect\models\artifacts\rtdetr\experiment\weights\last.pt, 66.2MB
Optimizer stripped from C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\runs\detect\models\artifacts\rtdetr\experiment\weights\best.pt, 66.2MB

Validating C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\runs\detect\models\artifacts\rtdetr\experiment\weights\best.pt...
Ultralytics 8.4.11  Python-3.12.3 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
rt-detr-l summary: 310 layers, 31,994,015 parameters, 0 gradients, 103.5 GFLOPs
                 Class     Images  Instances      Box(P   

# Visualización/comparación entrenamiento

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_yolo_vs_rtdetr_final():
    # Rutas absolutas proporcionadas
    yolo_csv = r"C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\models\artifacts\yolo_run\results.csv"
    rtdetr_csv = r"C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\runs\detect\models\artifacts\rtdetr\experiment\results.csv"
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # --- 1. GRÁFICA DE EVOLUCIÓN (Líneas) ---
    def load_and_plot(path, label, color):
        if os.path.exists(path):
            df = pd.read_csv(path)
            df.columns = df.columns.str.strip() # Elimina espacios en blanco de los nombres de columnas
            
            # Buscamos la columna de mAP50 (puede variar ligeramente el nombre)
            target_col = 'metrics/mAP50(B)'
            if target_col in df.columns:
                ax1.plot(df['epoch'], df[target_col], label=label, color=color, linewidth=2)
                print(f"DEBUG: Cargados datos de evolución para {label}")
            else:
                print(f"WARNING: No se encontró la columna '{target_col}' en {path}")
                print(f"Columnas disponibles: {df.columns.tolist()}")
        else:
            print(f"ERROR: Archivo no encontrado en {path}")

    load_and_plot(yolo_csv, 'YOLOv8s', '#1f77b4')
    load_and_plot(rtdetr_csv, 'RT-DETR-L', '#2ca02c')

    ax1.set_title('Evolución de Precisión (mAP50)')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('mAP50')
    ax1.set_ylim(0, 1)
    ax1.legend()
    ax1.grid(True, linestyle='--', alpha=0.6)

    # --- 2. GRÁFICA POR CLASE (Barras) ---
    # Usamos los datos finales de validación obtenidos de tus logs
    classes = ['stand', 'lying_down', 'foraging', 'drinking', 'rumination']
    # Datos de RT-DETR-L según tu log final (mAP50)
    rtdetr_vals = [0.904, 0.870, 0.835, 0.647, 0.684]
    # Datos de YOLOv8s (basados en tus resultados previos)
    yolo_vals = [0.921, 0.885, 0.878, 0.702, 0.721] 

    x = range(len(classes))
    width = 0.35
    ax2.bar([i - width/2 for i in x], yolo_vals, width, label='YOLOv8s', color='#1f77b4', alpha=0.7)
    ax2.bar([i + width/2 for i in x], rtdetr_vals, width, label='RT-DETR-L', color='#2ca02c', alpha=0.7)

    ax2.set_title('Precisión por Clase (mAP50)')
    ax2.set_xticks(x)
    ax2.set_xticklabels(classes, rotation=45)
    ax2.set_ylabel('mAP50')
    ax2.set_ylim(0, 1)
    ax2.legend()
    ax2.grid(axis='y', linestyle='--', alpha=0.6)

    plt.tight_layout()
    # Guardar en la carpeta de métricas para la auditoría
    metrics_path = r"C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\models\metrics\comparativa_modelos.png"
    os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
    plt.savefig(metrics_path)
    plt.show()

plot_yolo_vs_rtdetr_final()

DEBUG: Cargados datos de evolución para YOLOv8s
DEBUG: Cargados datos de evolución para RT-DETR-L


<Figure size 1600x600 with 2 Axes>

# PREDICT

In [19]:
# ===== LÓGICA PARA LA PREDICCIÓN =====
def predict():
    """
    Ejecuta inferencia sobre un video, realiza tracking y genera estadísticas.
    """
    config = load_config()
    root_path = config['root_path']

    # 1. Definir rutas de Entrada y Salida
    artifacts_dir = root_path / "models" / "artifacts" / "yolo_run" / "weights"
    model_path = artifacts_dir / "best.pt"

    # Busca un video en data/raw/videos (toma el primero que encuentre)
    videos_dir = root_path / config['paths']['raw_path'] / config['paths']['folders']['videos']
    video_files = list(videos_dir.glob("*.mp4")) + list(videos_dir.glob("*.avi"))

    if not video_files:
        print(f" Error: No se encontraron videos en {videos_dir}")
        return

    source_video = video_files[8] # Usamos el segundo video encontrado

    # Carpetas de salida
    predictions_dir = root_path / config['paths']['predictions_path']
    output_video_path = predictions_dir / f"pred_{source_video.name}"
    stats_csv_path = predictions_dir / f"stats_{source_video.stem}.csv"

    print(f"--- PREDICT: Iniciando Inferencia ---")
    print(f"Modelo: {model_path}")
    print(f"Video fuente: {source_video}")
    print(f"Salida video: {output_video_path}")
    print(f"Salida datos: {stats_csv_path}")

    # 2. Cargar Modelo Entrenado
    if not model_path.exists():
        print(" Error: No existe el modelo best.pt. Ejecuta train() primero.")
        return

    model = YOLO(str(model_path))

    # 3. Configurar Video
    cap = cv2.VideoCapture(str(source_video))
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Writer para guardar el video procesado
    video_writer = cv2.VideoWriter(
        str(output_video_path),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps, (w, h)
    )

    # 4. Estructuras para Datos de Comportamiento
    # Diccionario: {track_id: {class_id: frames_count}}
    behavior_data = defaultdict(lambda: defaultdict(int))
    class_names = model.names # {0: 'standing', 1: 'lying', ...}

    # 5. Bucle de Inferencia (Frame a Frame)
    frame_count = 0
    start_time = time.time()

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1

        # --- TRACKING CON YOLOv8 ---
        # persist=True es VITAL para el tracking (mantiene memoria entre frames)
        results = model.track(frame, persist=True, verbose=False, tracker="bytetrack.yaml")

        if results[0].boxes.id is not None:
            # Obtener datos: cajas, ids de track, y clases
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            cls_ids = results[0].boxes.cls.int().cpu().tolist()

            # Anotador visual
            annotated_frame = results[0].plot()

            # Lógica de Acumulación de Tiempo
            for track_id, cls_id in zip(track_ids, cls_ids):
                behavior_data[track_id][cls_id] += 1

                # Calcular tiempo actual en segundos (frames / fps)
                seconds = behavior_data[track_id][cls_id] / fps

                # --- LÓGICA DE ALERTAS (ANOMALÍA SIMPLE) ---
                # Ejemplo: Si una vaca lleva > 5 segundos "standing" (solo demo)
                # En producción esto sería horas.
                action_name = class_names[cls_id]
                label = f"ID:{track_id} {action_name} {seconds:.1f}s"

                # Dibujar info extra en el video
                # (YOLO ya dibuja cajas, aquí podríamos añadir alertas personalizadas)

        else:
            annotated_frame = frame # Si no detecta nada, guarda el frame original

        video_writer.write(annotated_frame)

        if frame_count % 30 == 0:
            print(f"Procesando frame {frame_count}...", end='\r')

    # 6. Finalización y Guardado de Datos
    cap.release()
    video_writer.release()

    # Exportar CSV final
    # Convertimos frames a segundos/minutos para el reporte
    rows = []
    for tid, actions in behavior_data.items():
        row = {'cow_id': tid}
        total_frames = 0
        for cid, count in actions.items():
            action_name = class_names[cid]
            row[f"{action_name}_sec"] = round(count / fps, 2)
            total_frames += count
        row['total_tracked_sec'] = round(total_frames / fps, 2)
        rows.append(row)

    df_stats = pd.DataFrame(rows)
    df_stats.to_csv(stats_csv_path, index=False)

    print(f"\n Procesamiento finalizado.")
    print(f"Video guardado: {output_video_path}")
    print(f"Reporte CSV: {stats_csv_path}")
    print(df_stats.head())

In [20]:
predict()

--- PREDICT: Iniciando Inferencia ---
Modelo: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\models\artifacts\yolo_run\weights\best.pt
Video fuente: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\raw\videos\106.mp4
Salida video: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\predictions\pred_106.mp4
Salida datos: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\predictions\stats_106.csv
Procesando frame 240...
 Procesamiento finalizado.
Video guardado: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\predictions\pred_106.mp4
Reporte CSV: C:\Bureau\Proyectos\DATAGIA\Modelo-CU11\data\predictions\stats_106.csv
   cow_id  stand_sec  total_tracked_sec  lying_down_sec  rumination_sec
0       1       10.0               10.0             NaN             NaN
1       2        NaN               10.0           10.00             NaN
2       3        NaN               10.0           10.00             NaN
3       4        NaN               10.0            5.96            4.04
4       5        NaN               10.0     

In [ ]:
# --- PREDICT CON LÓGICA MULTI-ESTADO ---
def predict():
    """
    Inferencia con fusión espacial para permitir múltiples estados por vaca.
    """
    config = load_config()
    root_path = config['root_path']

    # 1. Configuración de Rutas
    artifacts_dir = root_path / "models" / "artifacts" / "yolo_run" / "weights"
    model_path = artifacts_dir / "best.pt"

    videos_dir = root_path / config['paths']['raw_path'] / config['paths']['folders']['videos']
    if not videos_dir.exists():
        print(f" Error: Ruta no encontrada: {videos_dir}")
        return

    # Selección inteligente de video
    video_files = sorted(list(videos_dir.glob("*.mp4")) + list(videos_dir.glob("*.avi")))
    if not video_files:
        print(f" Error: No hay videos en {videos_dir}")
        return

    # Priorizamos "2.mp4" si existe (porque sabemos que tiene vacas)
    source_video = None
    for v in video_files:
        if v.name == "2.mp4":
            source_video = v
            break
    if source_video is None:
        source_video = video_files[0] if len(video_files) == 1 else video_files[1]

    predictions_dir = root_path / config['paths']['predictions_path']
    output_video_path = predictions_dir / f"pred_{source_video.name}"
    stats_csv_path = predictions_dir / f"stats_{source_video.stem}.csv"

    print(f"--- PREDICT (MULTI-LABEL MODE) ---")
    print(f"Modelo: {model_path}")
    print(f"Video: {source_video}")

    if not model_path.exists():
        print(" Error: No existe best.pt")
        return
    model = YOLO(str(model_path))

    cap = cv2.VideoCapture(str(source_video))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    video_writer = cv2.VideoWriter(
        str(output_video_path), 
        cv2.VideoWriter_fourcc(*'mp4v'), 
        fps, (w, h)
    )

    # Estructura: {cow_id: {class_name: frames}}
    # Usamos float para poder sumar tiempos parciales si quisiéramos, aunque aquí sumamos frames
    behavior_data = defaultdict(lambda: defaultdict(int))
    # Para contar el tiempo total que una vaca ha sido trackeada (independiente de la acción)
    tracking_duration = defaultdict(int) 

    class_names = model.names
    CONF_THRESHOLD = 0.70  # Tu requisito: Solo considerar estados con confianza > 0.70
    IOU_MERGE_THRESHOLD = 0.85 # Si dos cajas se superponen un 85%, son la misma vaca

    frame_count = 0

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        frame_count += 1

        # Tracking con umbral bajo para captar todo, luego filtramos nosotros
        results = model.track(frame, persist=True, verbose=False, tracker="bytetrack.yaml", conf=0.1)

        if results[0].boxes.id is not None:
            boxes_xyxy = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            cls_ids = results[0].boxes.cls.int().cpu().tolist()
            confs = results[0].boxes.conf.cpu().tolist()

            # 1. Agrupación Espacial (Merge)
            # Creamos grupos de detecciones que pertenecen a la misma vaca física
            # Estructura: [ {master_id: 1, boxes: [...]}, ... ]
            merged_cows = []

            # Lista temporal con toda la info
            current_detections = []
            for i in range(len(track_ids)):
                current_detections.append({
                    "id": track_ids[i],
                    "box": boxes_xyxy[i],
                    "cls": cls_ids[i],
                    "conf": confs[i]
                })

            # Algoritmo simple de fusión
            processed_indices = set()

            for i in range(len(current_detections)):
                if i in processed_indices:
                    continue

                # Esta vaca 'i' empieza un nuevo grupo (o es ella misma)
                cow_group = [current_detections[i]]
                processed_indices.add(i)

                # Buscamos otras cajas que se superpongan mucho con esta
                for j in range(i + 1, len(current_detections)):
                    if j in processed_indices:
                        continue

                    iou = box_iou(current_detections[i]["box"], current_detections[j]["box"])
                    if iou > IOU_MERGE_THRESHOLD:
                        cow_group.append(current_detections[j])
                        processed_indices.add(j)

                # 2. Procesar el grupo (La Vaca Física)
                # El ID principal será el menor del grupo (para mantener consistencia)
                master_id = min(d["id"] for d in cow_group)
                tracking_duration[master_id] += 1 # Un frame más que vemos a esta vaca

                # Recopilamos TODOS los estados que superen el umbral de 0.70
                active_states = set()
                for det in cow_group:
                    if det["conf"] > CONF_THRESHOLD:
                        state_name = class_names[det["cls"]]
                        active_states.add(state_name)

                        # Debug visual: pintar texto extra
                        # (Opcional: podrías pintar en el frame aquí)

                # 3. Acumular tiempos
                for state in active_states:
                    behavior_data[master_id][state] += 1

            annotated_frame = results[0].plot()
        else:
            annotated_frame = frame

        video_writer.write(annotated_frame)
        if frame_count % 30 == 0:
            print(f"Procesando frame {frame_count}...", end='\r')

    cap.release()
    video_writer.release()

    # --- GENERACIÓN DE CSV COMPLETO ---
    print("\nGenerando reporte multi-estado...")

    all_rows = []
    # Obtener todas las columnas posibles (todas las clases detectadas alguna vez o todas las del modelo)
    all_possible_states = list(class_names.values())

    for tid, states_dict in behavior_data.items():
        row = {'cow_id': tid}

        # Tiempo total trackeado (segundos)
        total_sec = round(tracking_duration[tid] / fps, 2)
        row['total_tracked_sec'] = total_sec

        # Rellenar columnas de estados
        for state_name in all_possible_states:
            frames_active = states_dict.get(state_name, 0)
            if frames_active > 0:
                row[f"{state_name}_sec"] = round(frames_active / fps, 2)
            else:
                row[f"{state_name}_sec"] = None # NaN para que quede limpio como pediste

        all_rows.append(row)

    df_stats = pd.DataFrame(all_rows)

    # Reordenar columnas para que quede bonito (cow_id, total, estados...)
    cols = ['cow_id', 'total_tracked_sec'] + [c for c in df_stats.columns if c not in ['cow_id', 'total_tracked_sec']]
    df_stats = df_stats[cols]

    df_stats.to_csv(stats_csv_path, index=False)

    print(f" Procesamiento finalizado.")
    print(f"Reporte CSV: {stats_csv_path}")
    if not df_stats.empty:
        print(df_stats.head())
    else:
        print(" DataFrame vacío.")